### Installing the requirements

In [ ]:
%%capture

!pip install --upgrade yt_dlp
!pip install pydub SpeechRecognition ffmpeg ffprobe googletrans==3.1.0a0 transformers

In [ ]:
import yt_dlp
import time
import re
import os
from pydub import AudioSegment
import speech_recognition as sr
import math
from tqdm import tqdm
from googletrans import Translator
from threading import Thread
from google.colab import files

### Downloading the audio (`url = video_link`)
> - Specify here the link of the video you want to transcribe.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import json

def extract_video_urls_from_json(json_file):
    with open(json_file, "r") as f:
        data = json.load(f)
    video_urls = [data['database'][video_id]['video_url'] for video_id in data['database']]
    return video_urls

# Example usage:
json_file = "/content/drive/MyDrive/youcookii_annotations_trainval.json"  # Change this to the path of your JSON file
video_urls = extract_video_urls_from_json(json_file)

In [ ]:
# url = "https://youtu.be/FLd00Bx4tOk?si=8E7s7_I4iZtigHeb"
print(len(video_urls))


1790


In [ ]:
class SplitWavAudioMubin():
    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = folder + filename
        self.audio = AudioSegment.from_wav(self.filepath)

    def get_duration(self):
        return self.audio.duration_seconds

    def single_split(self, from_min, to_min, split_filename):
        t1 = from_min * 60 * 1000
        t2 = to_min * 60 * 1000
        split_audio = self.audio[t1:t2]
        split_audio.export(split_filename, format="wav")

    def multiple_split(self, min_per_split):
        total_mins = math.ceil(self.get_duration() / 60)
        for i in range(0, total_mins, min_per_split):
            split_fn = str(i) + '_' + self.filename
            self.single_split(i, i+min_per_split, split_fn)
            if i == total_mins - min_per_split:
                print('All splited successfully')
        print('>>> Video duration: ' + str(self.get_duration()))

def split_audio(file_name):
    directory = "splitted files for: " + file_name
    os.mkdir(directory)
    os.chdir(directory)
    split_wav = SplitWavAudioMubin("/content/", file_name)
    split_wav.multiple_split(min_per_split=1)

In [ ]:
def speech_recognizer(files, frames, i):  ## This function recognizes speech in our WAV files
    texts = []
    recognizer = sr.Recognizer()

    for file in tqdm(files):
        with sr.AudioFile(file) as source:
            recorded_audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(
                recorded_audio,
                language="en-US"  ## Replace with language keyword
            )
            texts.append(text)
        except Exception as ex:
            print(ex)
    result = ""
    for text in texts:
        result += " " + text
    frames[i] = result
    return result

def split_files(files, n_batches): ## This function split the files evenly between the threads we have.
    k, m = divmod(len(files), n_batches)
    return list(files[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n_batches))

def main(files, n_batches=8, verbose=True): ## By default, the maximum capacity of threads supported in collab is 8
    threads = [None]*n_batches
    frames = [None]*n_batches
    batches = split_files(files, n_batches)
    start = 0
    for i in range(len(batches)):
        if i>0:
            start_index=len(batches[i-1])
        else:
            start_index = 0
        t = Thread(target=speech_recognizer, args=(batches[i], frames, i))
        threads[i] = t
        t.start()
    for t in threads:
        t.join()
    return frames

In [ ]:
def preprocess_video(url):
  # ydl_opts={}
  # with yt_dlp.YoutubeDL(ydl_opts) as ydl:
  #   info_dict = ydl.extract_info(url, download=False)
  # video_title = info_dict['title']
  try:
    ydl_opts = {}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      info_dict = ydl.extract_info(url, download=False)
    video_title = info_dict['title']
  except yt_dlp.DownloadError:
    print(f"Video at URL {url} is not available or cannot be downloaded.")
    return None, None

  video_name = re.sub('[\\\\/*?:"<>|]', '', video_title)
  name = video_name
  ydl_opts = {
     'format': 'm4a/bestaudio/best',
         'noplaylist': True,
         'continue_dl': True,
         'outtmpl': f'./{name}.wav',
         'postprocessors': [{
             'key': 'FFmpegExtractAudio',
             'preferredcodec': 'wav',
             'preferredquality': '192',
         }],
         'geobypass':True,
         'ffmpeg_location':'/usr/bin/ffmpeg'
  }
  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download(url)
  print("check point 1")
  file_name = "{}.wav.wav".format(video_name)
  directory = "splitted files for: " + file_name
  if os.path.exists(directory):
        print(f"Directory '{directory}' already exists. Skipping...")
        return None, None
  os.mkdir(directory)
  os.chdir(directory)
  split_wav = SplitWavAudioMubin("/content/", file_name)
  split_wav.multiple_split(min_per_split=1)
  print("check point 2")
  search_dir = "./"
  files = filter(os.path.isfile, os.listdir(search_dir))

  files = [os.path.join(search_dir, f) for f in files] # add path to each file
  files.sort(key=lambda x: os.path.getmtime(x))
  print("check point 3")
  frames = main(files)
  print("check point 4")
  result = ""
  for text in frames:
    result += " " + text
  os.chdir("../")
  text_file = open("Transcription_"+ file_name[:-4] +".txt", "w")
  text_file.write(result)
  text_file.close()
  return result, video_name
  # print(result)

In [ ]:
output_file = "/content/drive/MyDrive/output3.txt"
print()

In [ ]:
count = 0
urls_to_process = video_urls[80:]
print(len(urls_to_process))

1710


In [ ]:
with open(output_file, "a") as outfile:
  for url in video_urls:
    result, video_id = preprocess_video(url)
    outfile.write(f"Video ID: {video_id}\n\nResult: {result}\n\n\n\n\n\n\n")
    print(f"\n\n\n\n\n\nCount:{count}")
    count += 1

[youtube] Extracting URL: https://www.youtube.com/watch?v=GLd3aX16zBg
[youtube] GLd3aX16zBg: Downloading webpage
[youtube] GLd3aX16zBg: Downloading ios player API JSON
[youtube] GLd3aX16zBg: Downloading android player API JSON
[youtube] GLd3aX16zBg: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=GLd3aX16zBg
[youtube] GLd3aX16zBg: Downloading webpage
[youtube] GLd3aX16zBg: Downloading ios player API JSON
[youtube] GLd3aX16zBg: Downloading android player API JSON
[youtube] GLd3aX16zBg: Downloading m3u8 information
[info] GLd3aX16zBg: Downloading 1 format(s): 140
[download] Destination: ./Grilled Cheese Recipe - How to make a White Trash Grilled Cheese Sandwich.wav
[download] 100% of    3.73MiB in 00:00:55 at 68.86KiB/s  
[FixupM4a] Correcting container of "./Grilled Cheese Recipe - How to make a White Trash Grilled Cheese Sandwich.wav"
[ExtractAudio] Destination: ./Grilled Cheese Recipe - How to make a White Trash Grilled Cheese Sandwich.wav.wav
De

  0%|          | 0/1 [00:00<?, ?it/s]
0it [00:00, ?it/s]


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]



0it [00:00, ?it/s]




  0%|          | 0/1 [00:00<?, ?it/s]





  0%|          | 0/1 [00:00<?, ?it/s]






0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]



100%|██████████| 1/1 [00:00<00:00,  9.32it/s]


100%|██████████| 1/1 [00:02<00:00,  2.23s/it]








100%|██████████| 1/1 [00:04<00:00,  4.30s/it]




100%|██████████| 1/1 [00:15<00:00, 15.26s/it]

...








100%|██████████| 1/1 [00:05<00:00,  5.18s/it]





100%|██████████| 1/1 [00:11<00:00, 11.06s/it]




100%|██████████| 1/1 [00:20<00:00, 21.00s/it]


100%|██████████| 1/1 [00:30<00:00, 30.74s/it]

check point 4






Count:29
[youtube] Extracting URL: https://www.youtube.com/watch?v=frCFxOt9390
[youtube] frCFxOt9390: Downloading webpage


[youtube] frCFxOt9390: Downloading ios player API JSON
[youtube] frCFxOt9390: Downloading android player API JSON
[youtube] frCFxOt9390: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=frCFxOt9390
[youtube] frCFxOt9390: Downloading webpage
[youtube] frCFxOt9390: Downloading ios player API JSON
[youtube] frCFxOt9390: Downloading android player API JSON
[youtube] frCFxOt9390: Downloading m3u8 information
[info] frCFxOt9390: Downloading 1 format(s): 140
[download] Destination: ./How To Make Lamb Vindaloo - Indian Video Recipe.wav
[download] 100% of    4.27MiB in 00:00:00 at 5.34MiB/s   
[FixupM4a] Correcting container of "./How To Make Lamb Vindaloo - Indian Video Recipe.wav"
[ExtractAudio] Destination: ./How To Make Lamb Vindaloo - Indian Video Recipe.wav.wav
Deleting original file ./How To Make Lamb Vindaloo - Indian Video Recipe.wav (pass -k to keep)
check point 1
All splited successfully
>>> Video duration: 276.43356009070294
check point 2
check 

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


  0%|          | 0/1 [00:00<?, ?it/s]




0it [00:00, ?it/s]






0it [00:00, ?it/s]




  0%|          | 0/1 [00:00<?, ?it/s]





0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [ ]:
files.download(output_file)